In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.special import expit
from sklearn.decomposition import PCA


In [2]:
def abrirImagenesEscaladas( carpeta, escala=200):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [3]:
def balancear_datos(imagenes_entrenamiento):
    imagenes_entrenamiento_balanceadas = abrirImagenesEscaladas(imagenes_entrenamiento)
    return imagenes_entrenamiento_balanceadas

In [4]:
x_sanos = np.array(balancear_datos("train_mini/normal"))
x_insanos = np.array(balancear_datos("train_mini/neumonia"))

KeyboardInterrupt: 

In [5]:
TOLERANCIA = 0.0001

In [6]:
def f(x,w,b):
    return (np.tanh((np.dot(w, x)) + b) + 1) / 2

In [8]:
#sum((f-d)*1-tan(w*i+b)**2)*i
def dW(x,w,b,d):
    y = (np.tanh(np.dot(x,w)+b)+1)/2
    return np.dot((y-d)*((1-np.tanh(np.dot(x,w)+b))**2),x) 


In [ ]:
#sum((f-d)*1-tan(w*i+b)**2)
def dB(x,w,b,d):
    y = (np.tanh(np.dot(x,w)+b)+1)/2
    return (y-d)*((1-np.tanh(np.dot(x,w)+b))**2)


In [9]:
def gradient_descent(datos_sanos,datos_neum,learning_rate, iter=1000):
    # x = normalizar(x)
    w = np.zeros(x.shape[1])
    b = 0
    #grad_w = 0
    #grad_b = 0


    for i in range(0,iter):
        grad_w = 0
        grad_b = 0
        # Cuando los pacientes son sanos
        d = 0
        for i in len(datos_sanos):
            grad_w += dW(datos_sanos[i], w, b, d)
            grad_b += dB(datos_sanos[i], w, b, d)

        #Cuando los pacientes tienen neum
        d=1
        for i in len(datos_neum):
            grad_w += dW(datos_neum[i], w, b, d)
            grad_b += dB(datos_neum[i], w, b, d)

        # Actualización de los parámetros
        w_sig = w - learning_rate * grad_w
        b_sig = b - learning_rate * grad_b
        print(w_sig)

        # Calcular el costo actual (para monitorear el progreso)
        err1 = L(x,y,w,b)
        err2 = L(x,y,w_sig,b_sig)
    
        print(f'Iteracion {i}, Minimo error: {err1}')
        
        if abs(err1-err2) < TOLERANCIA:
            print(f'Convergencia en iteracion {i}')
            break

        # print(f'w:{w}, w_sig:{w_sig}')

        w = w_sig
        b = b_sig

    return w, b